In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import pandas as pd
import talib as ta
import pandas_ta as pandas_ta
import numpy as np
import matplotlib.pyplot as plt
from backtesting import Strategy
import plotly.express as px
from datetime import datetime
import MetaTrader5 as mt5

import random
random.seed(42)

from backbone.utils.general_purpose import diff_pips

In [3]:
symbols_path = './backbone/data/backtest/symbols/USDCAD.csv'
df = pd.read_csv(symbols_path)
df

,Date,Open,High,Low,Close,Volume
0,2021-08-02 00:00:00,1.24668,1.24721,1.24668,1.24719,118.27
1,2021-08-02 00:01:00,1.24722,1.24726,1.24709,1.24725,45.03
2,2021-08-02 00:02:00,1.24721,1.24721,1.24694,1.24713,87.82
3,2021-08-02 00:03:00,1.24714,1.24743,1.24713,1.24741,61.90
4,2021-08-02 00:04:00,1.24739,1.24739,1.24706,1.24720,94.39
...,...,...,...,...,...,...
897525,2024-01-01 23:55:00,1.32462,1.32462,1.32462,1.32462,2.40
897526,2024-01-01 23:56:00,1.32463,1.32465,1.32441,1.32443,43.27
897527,2024-01-01 23:57:00,1.32445,1.32450,1.32435,1.32436,23.85
897528,2024-01-01 23:58:00,1.32435,1.32436,1.32435,1.32435,9.60


In [4]:
train_start = '2021-08-01'
train_end = '2021-11-01'

test_start = '2021-10-01'
test_end = '2021-11-01'

wfo_start = '2022-01-01'
wfo_end = '2022-08-01'

train_data = df[(df.Date > train_start) & (df.Date < train_end)]
test_data = df[(df.Date > test_start) & (df.Date < test_end)]
wfo_data = df[(df.Date > wfo_start) & (df.Date < wfo_end)]

train_data.loc[:, 'Date'] = pd.to_datetime(train_data.Date)
test_data.loc[:, 'Date'] = pd.to_datetime(test_data.Date)
wfo_data.loc[:, 'Date'] = pd.to_datetime(wfo_data.Date)

df.loc[:, 'Date'] = pd.to_datetime(df.Date)

train_data = train_data.set_index('Date')
test_data = test_data.set_index('Date')
wfo_data = wfo_data.set_index('Date')

df = df.set_index('Date')


# Percentile_exit_ultimate Strategy

## Test Exit: Monkey Entry

In [22]:

trade_frecuency = 0.001
long_frecuency = 0.5

def random_boolean(prob_true=0.5):
    return random.choices([True, False], weights=[prob_true, 1 - prob_true], k=1)[0]

class PercentileExit(Strategy):
    pip_size = 0.0001
    n_bars = 5
    sl_pips = 15
    risk = 2

    def init(self):
        self.random = None

    def next(self):
        actual_close = self.data.Close[-1]
       
        if self.position:
            percentile = np.percentile(self.data.Close[-self.n_bars:-1], q=50)
            
            if self.position.is_long and actual_close < percentile:
                self.position.close()
                
            elif self.position.is_short and actual_close > percentile:
                self.position.close()

        else: 
            trade = random_boolean(prob_true=trade_frecuency)

            if trade:
                long = random_boolean(prob_true=long_frecuency)

                if long:
                    account_risk = self.equity * (self.risk / 100)
                    units = round(account_risk / (self.pip_size * self.sl_pips))
                    self.buy(size=units)

                else:
                    account_risk = self.equity * (self.risk / 100)
                    
                    # Calculate lot size in units
                    units = round(account_risk / (self.pip_size * self.sl_pips))
                    self.sell(size=units)


metrics = pd.DataFrame()
n_bars = [3, 5, 7, 9, 12]

for x in range(0, 10):
    for n in n_bars:
        bt_train = Backtest(
            train_data, 
            PercentileExit, 
            cash=15_000, 
            margin=1/30
        )

        stats = bt_train.run(
            n_bars=n
        )

        # bt_train.plot(filename='./RsiBBands.html', resample=False)

        equity = stats['Equity Final [$]']
        return_ = stats['Return [%]']
        sharpe_ratio = stats['Sharpe Ratio']

        actual_metrics = pd.DataFrame(
            {
                'n_bars': [n],
                'equity':[equity], 
                'return_':[return_], 
                'sharpe_ratio':[sharpe_ratio], 
            }
        )

        metrics = pd.concat([metrics, actual_metrics])

metrics

,n_bars,equity,return_,sharpe_ratio
0,3,15162.44568,1.082971,0.627857
0,5,14994.65018,-0.035665,0.050283
0,7,15620.72111,4.138141,1.726091
0,9,14626.44332,-2.490378,0.000000
0,12,14974.78931,-0.168071,0.095494
0,3,14860.78891,-0.928074,0.000000
0,5,15621.87250,4.145817,2.171422
0,7,14542.30483,-3.051301,0.000000
0,9,14595.43706,-2.697086,0.000000
0,12,14801.09766,-1.326016,0.000000


In [23]:
metrics.groupby(by='n_bars').median()

,equity,return_,sharpe_ratio
n_bars,,,
3,15104.371345,0.695809,0.560107
5,14878.051310,-0.812991,0.000000
7,14634.121705,-2.439189,0.000000
9,14622.266695,-2.518222,0.000000
12,14780.636895,-1.462421,0.000000


In [24]:
bt_train.plot(filename='./RsiBBands.html', resample=False)


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p2456', ...)

In [37]:
class PercentileExit(Strategy):
    pip_size = 0.0001
    n_bars = 5
    sl_pips = 15
    risk = 2

    def init(self):
        self.random = None
        self.ultimate = self.I(ta.ULTOSC, self.data.High, self.data.Low, self.data.Close)

    def next(self):
        actual_close = self.data.Close[-1]
       
        if self.position:
            percentile = np.percentile(self.data.Close[-self.n_bars:-1], q=50)
            
            if self.position.is_long and actual_close < percentile:
                self.position.close()
                
            elif self.position.is_short and actual_close > percentile:
                self.position.close()

        else: 
            high_osc = max(self.ultimate[-self.n_bars: -1])
            low_osc = min(self.ultimate[-self.n_bars: -1])

            if self.ultimate < 20:
                account_risk = self.equity * (self.risk / 100)
                units = round(account_risk / (self.pip_size * self.sl_pips))
                self.buy(size=units)

            elif self.ultimate > 80:
                account_risk = self.equity * (self.risk / 100)
                
                # Calculate lot size in units
                units = round(account_risk / (self.pip_size * self.sl_pips))
                self.sell(size=units)


metrics = pd.DataFrame()
n_bars = [12, 20, 40, 50, 60]

for n in n_bars:
    bt_train = Backtest(
        train_data, 
        PercentileExit, 
        cash=15_000, 
        margin=1/30
    )

    stats = bt_train.run(
        n_bars=n
    )

    actual_metrics = pd.DataFrame([stats])

    metrics = pd.concat([metrics, actual_metrics])

metrics

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],Sharpe Ratio,Sortino Ratio,Calmar Ratio,Max. Drawdown [%],Avg. Drawdown [%],Max. Drawdown Duration,Avg. Drawdown Duration,# Trades,Win Rate [%],Best Trade [%],Worst Trade [%],Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,_strategy,_equity_curve,_trades
0,2021-08-02,2021-10-31 23:59:00,90 days 23:59:00,2.242885,15639.35456,15729.06106,4.262364,-0.710397,14.931609,10.885879,1.371649,2.525260,2.591559,-5.761632,-0.353942,75 days 00:05:00,2 days 15:29:00,701,41.797432,0.063871,-0.069173,0.000380,0 days 00:48:00,0 days 00:03:00,1.119065,0.000381,0.834008,PercentileExit(n_bars=12),Equity DrawdownPct ...,Size EntryBar ExitBar EntryPrice Ex...
0,2021-08-02,2021-10-31 23:59:00,90 days 23:59:00,2.103246,15558.64260,15685.34649,3.724284,-0.710397,13.026312,10.414237,1.250818,2.214614,2.497055,-5.216671,-0.384212,74 days 01:22:00,3 days 02:15:00,714,40.896359,0.062990,-0.077611,0.000327,0 days 00:40:00,0 days 00:02:00,1.106839,0.000327,0.735836,PercentileExit(n_bars=20),Equity DrawdownPct D...,Size EntryBar ExitBar EntryPrice Ex...
0,2021-08-02,2021-10-31 23:59:00,90 days 23:59:00,2.097834,15733.77094,15799.11427,4.891806,-0.710397,17.188423,11.143459,1.542467,3.010480,3.192875,-5.383369,-0.329560,74 days 02:31:00,2 days 16:36:00,722,40.997230,0.062990,-0.069173,0.000419,0 days 01:02:00,0 days 00:02:00,1.138066,0.000420,0.962648,PercentileExit(n_bars=40),Equity DrawdownPct ...,Size EntryBar ExitBar EntryPrice Ex...
0,2021-08-02,2021-10-31 23:59:00,90 days 23:59:00,2.238556,15142.39364,15285.63624,0.949291,-0.710397,3.545262,8.697409,0.407623,0.588226,0.656666,-5.398882,-0.499916,80 days 05:11:00,5 days 00:00:00,723,39.557400,0.059937,-0.069173,0.000098,0 days 00:58:00,0 days 00:03:00,1.031651,0.000098,0.194981,PercentileExit(n_bars=50),Equity DrawdownPct ...,Size EntryBar ExitBar EntryPrice Ex...
0,2021-08-02,2021-10-31 23:59:00,90 days 23:59:00,2.315411,15043.71222,15496.46573,0.291415,-0.710397,1.380963,9.027764,0.152968,0.211102,0.210691,-6.554459,-0.466441,76 days 23:04:00,3 days 11:04:00,731,38.850889,0.106256,-0.069173,0.000045,0 days 01:27:00,0 days 00:03:00,1.014537,0.000046,0.056342,PercentileExit(n_bars=60),Equity DrawdownPct ...,Size EntryBar ExitBar EntryPrice Ex...


In [32]:
pd.set_option('display.max_columns', None)


In [33]:
metrics

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],Sharpe Ratio,Sortino Ratio,Calmar Ratio,Max. Drawdown [%],Avg. Drawdown [%],Max. Drawdown Duration,Avg. Drawdown Duration,# Trades,Win Rate [%],Best Trade [%],Worst Trade [%],Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,_strategy,_equity_curve,_trades
0,2021-08-02,2021-10-31 23:59:00,90 days 23:59:00,0.340979,15067.39406,15119.58813,0.449294,-0.710397,1.502581,1.840772,0.816278,1.263171,1.450709,-1.035757,-0.429089,47 days 16:09:00,15 days 00:29:00,38,39.473684,0.027749,-0.028470,0.000697,0 days 00:28:00,0 days 00:09:00,1.246583,0.000697,0.442874,PercentileExit(n_bars=3),Equity DrawdownPct ...,Size EntryBar ExitBar EntryPrice Exi...
0,2021-08-02,2021-10-31 23:59:00,90 days 23:59:00,0.372371,15100.95436,15167.46565,0.673029,-0.710397,2.234814,2.541696,0.879261,1.547800,1.635446,-1.366486,-0.321723,54 days 00:28:00,10 days 00:17:00,38,36.842105,0.045906,-0.028470,0.001058,0 days 00:29:00,0 days 00:10:00,1.308464,0.001059,0.489152,PercentileExit(n_bars=5),Equity DrawdownPct ...,Size EntryBar ExitBar EntryPrice Exi...
0,2021-08-02,2021-10-31 23:59:00,90 days 23:59:00,0.396185,15000.06126,15106.35979,0.000408,-0.710397,0.044421,2.950992,0.015053,0.030215,0.034445,-1.289619,-0.364288,60 days 16:27:00,12 days 20:59:00,39,23.076923,0.071234,-0.028470,0.000012,0 days 00:29:00,0 days 00:10:00,1.003322,0.000013,0.000252,PercentileExit(n_bars=7),Equity DrawdownPct ...,Size EntryBar ExitBar EntryPrice Exi...
0,2021-08-02,2021-10-31 23:59:00,90 days 23:59:00,0.408093,14988.10541,15082.25517,-0.079297,-0.710397,-0.212974,2.913320,0.000000,0.000000,0.000000,-1.485763,-0.413792,61 days 16:12:00,15 days 00:29:00,40,22.500000,0.075191,-0.028470,-0.000107,0 days 00:36:00,0 days 00:10:00,0.970975,-0.000106,-0.049494,PercentileExit(n_bars=9),Equity DrawdownPct ...,Size EntryBar ExitBar EntryPrice Exi...
0,2021-08-02,2021-10-31 23:59:00,90 days 23:59:00,0.437319,14892.96560,15039.92388,-0.713563,-0.710397,-2.244947,4.817853,0.000000,0.000000,0.000000,-2.847471,-0.821747,88 days 18:14:00,22 days 12:43:00,40,20.000000,0.136927,-0.070179,-0.001040,0 days 00:36:00,0 days 00:11:00,0.805562,-0.001037,-0.266541,PercentileExit(n_bars=12),Equity DrawdownPct D...,Size EntryBar ExitBar EntryPrice Exi...


In [36]:
bt_train.plot(filename='./RsiBBands.html', resample=False)


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p5731', ...)

## Full system

In [ ]:
import itertools

class VwapRsiFull(Strategy):
    pip_size = 0.0001
    sl_pips = 10
    rr = 1
    risk = 1

    n_candles = 7 
    distance = 1

    rsi_up_threshold=70
    rsi_down_threshold=30


    def init(self):
        self.vwap = self.I(
            pandas_ta.vwap, 
            pd.Series(self.data.High, index=self.data.index),
            pd.Series(self.data.Low, index=self.data.index),
            pd.Series(self.data.Close, index=self.data.index),
            pd.Series(self.data.Volume, index=self.data.index),
        ) 

        self.ema_50 = self.I(ta.EMA, self.data.Close, timeperiod=200) 
        
        self.rsi = self.I(ta.RSI, self.data.Close, 14)


    def next(self):
        actual_close = self.data.Close[-1]
       
        if self.position:
            pass

        else: 

            actual_vwap = self.vwap[-1]
            distance_vwap = diff_pips(actual_vwap, actual_close, pip_value=self.pip_size, absolute=True)

            n_candles_under_vwap = True
            n_candles_up_vwap = True

            for x in range(1, self.n_candles):
                if self.data.Close[-x] > self.vwap[-x]:
                    n_candles_under_vwap = False

                if self.data.Close[-x] < self.vwap[-x]:
                    n_candles_up_vwap = False

            if distance_vwap <= self.distance and n_candles_up_vwap: 
                sl = actual_close - self.sl_pips * self.pip_size
                tp = actual_close + self.rr * self.sl_pips * self.pip_size
                
                account_risk = self.equity * (self.risk / 100)
                units = round(account_risk / (self.pip_size * self.sl_pips))
                self.buy(sl=sl, size=units, tp=tp)

            elif distance_vwap <= self.distance and n_candles_under_vwap: 
                sl = actual_close + self.sl_pips * self.pip_size
                tp = actual_close - self.rr * self.sl_pips * self.pip_size

                account_risk = self.equity * (self.risk / 100)
                
                # Calculate lot size in units
                units = round(account_risk / (self.pip_size * self.sl_pips))
                self.sell(sl=sl, size=units, tp=tp)


bt_train = Backtest(
    train_data, 
    VwapRsiFull, 
    commission=0.0002,
    cash=15_000, 
    margin=1/30
)


stats = bt_train.run(rr=2, sl_pips=15)

# stats = bt_train.optimize(
#     distance=[1, 2, 3],
#     n_candles=[6, 8,  10, 12],
#     sl_pips=[5, 8, 12, 15],
#     rsi_up_threshold=[60, 70, 80],
#     rsi_down_threshold=[20, 30, 40],
# )

bt_train.plot(filename='./RsiBBands.html', resample=False)

stats


In [ ]:
stats._strategy.__dict__

# WFO

In [1]:
import pickle
from wfo_utils.utils import walk_forward

lookback_bars = 60*1440
validation_bars = 7*1440
warmup_bars = 1440 * 4 * 14 

params = {
    'n': [3, 5, 7, 10],
    'adx_period': [3, 5, 7, 10],
    'maximize': 'Return [%]' 
}


stats = walk_forward(
    BreakoutTwist,
    wfo_data, 
    lookback_bars=lookback_bars,
    validation_bars=validation_bars,
    warmup_bars=warmup_bars, 
    params=params,
    commission=0.0002, 
    margin=1/30, 
    cash=15_000
)

NameError: name 'BreakoutTwist' is not defined

In [ ]:
stats

In [ ]:
stats = pd.DataFrame(stats)
stats['Win Rate [%]'].mean()

In [ ]:
plot_stats(df, stats[1], BreakoutTwist, plot=True)

In [ ]:
plot_full_equity_curve(df, stats, warmup_bars = warmup_bars,
                       lookback_bars = lookback_bars, overlay_price = True)

In [ ]:
trades = test_stats._trades.groupby(by=['ExitTime']).agg({'PnL':['sum','count'], 'Duration':'max'})
trades.columns = trades.columns.droplevel(0)
trades = trades.reset_index().rename(columns={'count':'ammount_trades'})
trades = trades.rename(columns={'sum':'profit'})
trades = trades.rename(columns={'max':'minutes_in_trade'})
trades

In [ ]:
trades.minutes_in_trade.describe()
